<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [65]:
# Import libraries here.

import pandas as pd
import numpy as np
import os

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [99]:
# Create an output folder.
path = '/Users/ericschultz/GA/submissions/labs/2.02-lab-excel-madness-master/output/'
os.mkdir(path)

# Create a files variable that contains all of our data files.

FileExistsError: [Errno 17] File exists: '/Users/ericschultz/GA/submissions/labs/2.02-lab-excel-madness-master/output/'

In [112]:
files = os.listdir('/Users/ericschultz/GA/submissions/labs/2.02-lab-excel-madness-master/data/')

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [113]:
# Read in data from your city from January 1st.

df = pd.read_excel('./data/Jan 1.xlsx', sheet_name='Washington, DC')

### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [114]:
df['price_usd']=df['price_eu']*1.1
df['weight_lb']=df['weight_kg']*2.2

### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [115]:
plu = pd.read_csv("./plu-codes.csv")

In [116]:
# Merge prodcode(df) to plu-code(csv)

df = pd.merge(df, plu, left_on='prodcode', right_on='plu_code')

### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [117]:
df.drop(columns=['price_eu', 'weight_kg'])

,prodcode,quantity,price_usd,weight_lb,product,plu_code
0,4011,182,1.578451,2.476043,Banana,4011
1,4011,316,0.758458,20.002665,Banana,4011
2,4011,438,2.055082,8.506290,Banana,4011
3,4011,338,3.156677,9.923875,Banana,4011
4,4011,362,1.747347,7.281211,Banana,4011
...,...,...,...,...,...,...
144,4159,298,2.925401,21.696434,Onion,4159
145,4309,300,2.543354,19.535528,Lychee,4309
146,4309,114,2.655647,17.466717,Lychee,4309
147,4309,245,3.275853,20.809142,Lychee,4309


### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [118]:
# A function? 
# for csv in files:
   # frame = pd.read_csv(csv)
   # frame['date'] = os.path.basename(csv)
   # data.append(frame)
    
df['date'] = os.path.basename('./data/Jan 1.xlsx')

# I want to add this to it: .str.rstrip('.xlsx')
# ...maybe at the end?
df

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb,product,plu_code,date
0,4011,1.434955,1.125474,182,1.578451,2.476043,Banana,4011,Jan 1.xlsx
1,4011,0.689507,9.092120,316,0.758458,20.002665,Banana,4011,Jan 1.xlsx
2,4011,1.868257,3.866495,438,2.055082,8.506290,Banana,4011,Jan 1.xlsx
3,4011,2.869706,4.510852,338,3.156677,9.923875,Banana,4011,Jan 1.xlsx
4,4011,1.588497,3.309641,362,1.747347,7.281211,Banana,4011,Jan 1.xlsx
...,...,...,...,...,...,...,...,...,...
144,4159,2.659456,9.862015,298,2.925401,21.696434,Onion,4159,Jan 1.xlsx
145,4309,2.312140,8.879785,300,2.543354,19.535528,Lychee,4309,Jan 1.xlsx
146,4309,2.414224,7.939417,114,2.655647,17.466717,Lychee,4309,Jan 1.xlsx
147,4309,2.978048,9.458701,245,3.275853,20.809142,Lychee,4309,Jan 1.xlsx


### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [119]:
# Translate and combine above steps into function for an individual csv

def process_data(file, city):
    prod = pd.read_csv('plu-codes.csv')
    df_new = pd.read_excel(file, sheet_name = city)
    df_new['price_usd'] = df_new['price_eu'] * 1.1
    df_new['weight_lb'] = df_new['weight_kg'] * 2.2
    df_cleaned = df_new.merge(prod, left_on='prodcode', right_on='plu_code')
    df_cleaned.drop(columns = ['price_eu','weight_kg','plu_code'])
    df_cleaned['date'] = os.path.basename(file)
    df_cleaned['date'] = [i.split('.')[0] for i in df_cleaned['date']]
    
    return df_cleaned

Test your function out on a new file and city!

In [120]:
process_data('./data/Jan 10.xlsx', 'Chicago')

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb,product,plu_code,date
0,4134,0.949925,7.503535,320,1.044917,16.507777,Apple (Gala),4134,Jan 10
1,4134,1.575662,8.477382,241,1.733228,18.650240,Apple (Gala),4134,Jan 10
2,4134,1.265133,5.741626,278,1.391646,12.631576,Apple (Gala),4134,Jan 10
3,4134,2.470747,1.589343,172,2.717821,3.496554,Apple (Gala),4134,Jan 10
4,4134,1.278187,5.416461,157,1.406006,11.916214,Apple (Gala),4134,Jan 10
...,...,...,...,...,...,...,...,...,...
183,4309,1.712223,7.531195,320,1.883445,16.568628,Lychee,4309,Jan 10
184,4309,2.090545,8.496443,356,2.299599,18.692174,Lychee,4309,Jan 10
185,4309,2.050277,8.796285,327,2.255304,19.351828,Lychee,4309,Jan 10
186,4309,1.988726,6.965311,194,2.187598,15.323685,Lychee,4309,Jan 10


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [122]:
january = [process_data('/Users/ericschultz/GA/submissions/labs/2.02-lab-excel-madness-master/data/' + file, 'Washington, DC') for file in files]

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'\x0cEric Sc'

### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [ ]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [ ]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!

# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).